In [47]:
import os
import random
import shutil
import cv2
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras.layers.normalization import BatchNormalization
from tensorflow.keras.models import load_model

import matplotlib.image as mpimg
from sklearn.preprocessing import LabelBinarizer

In [12]:
proportion = 0.1
BATCH_SIZE = 20
SIZE = 150
default_image_size = tuple((128,128))
EPOCH = 5

In [ ]:
# Get train and val data
data_train = os.listdir('tomato/train_lite/')
data_val = os.listdir('tomato/test_lite/')

In [ ]:
# Get a proportion of whole data
for folder in data_train:
    i = 0
    for data in os.listdir('tomato/val/'+folder):
        if(i < proportion * len(os.listdir('tomato/val/'+folder))):
            shutil.copyfile('tomato/val/'+folder+'/'+data, 'tomato/test_lite/'+folder+'/'+data)
            i += 1

In [13]:
base_dir = 'tomato'

train_dir = 'tomato/train_lite'
test_dir = 'tomato/test_lite'

In [14]:
# All images will be rescaled by 1./255.
train_datagen = ImageDataGenerator( rescale = 1.0/255. )
test_datagen  = ImageDataGenerator( rescale = 1.0/255. )


train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=BATCH_SIZE,
                                                    class_mode='categorical',
                                                    target_size=(150, 150))     

test_generator =  test_datagen.flow_from_directory(test_dir,
                                                         batch_size=BATCH_SIZE,
                                                         class_mode  = 'categorical',
                                                         target_size = (150, 150))

Found 1000 images belonging to 10 classes.
Found 100 images belonging to 10 classes.


In [15]:
#chanDim = -1
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    tf.keras.layers.Conv2D(128, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(3,3),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2), 
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'), 
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'), 
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(), 
    # 512 neuron hidden layer
    tf.keras.layers.Dense(1024, activation='relu'), 
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('cats') and 1 for the other ('dogs')
    tf.keras.layers.Dense(10, activation='softmax')  
])

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 128)     3584      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 49, 49, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 47, 47, 128)       147584    
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 45, 45, 128)       147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 22, 22, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 20, 20, 128)       147584    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 18, 18, 128)       1

In [17]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics = ['accuracy'])

In [18]:
#aug = ImageDataGenerator(fill_mode="nearest")
history = model.fit(train_generator,
                    validation_data=test_generator,
                    steps_per_epoch=len(train_generator),
                    validation_steps =len(test_generator),
                    epochs=EPOCH)

Epoch 1/5
50/50 [==============================] - 109s 2s/step - loss: 2.3106 - accuracy: 0.0990 - val_loss: 2.3013 - val_accuracy: 0.1000
Epoch 2/5
50/50 [==============================] - 69s 1s/step - loss: 2.1305 - accuracy: 0.1920 - val_loss: 1.9153 - val_accuracy: 0.3600
Epoch 3/5
50/50 [==============================] - 71s 1s/step - loss: 1.8376 - accuracy: 0.3330 - val_loss: 1.8432 - val_accuracy: 0.3900
Epoch 4/5
50/50 [==============================] - 75s 1s/step - loss: 1.5808 - accuracy: 0.4140 - val_loss: 1.6473 - val_accuracy: 0.4000
Epoch 5/5
50/50 [==============================] - 77s 2s/step - loss: 1.4275 - accuracy: 0.4870 - val_loss: 1.3097 - val_accuracy: 0.5600


In [19]:
model.save('modelAsli/model1.h5')

In [50]:
img=image.load_img('sehat.jpg', target_size=(150, 150))
x=image.img_to_array(img)
x=np.expand_dims(x, axis=0)
images = np.vstack([x])

model.predict(images, batch_size=BATCH_SIZE)

array([[0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        1.0000000e+00, 2.1847597e-22]], dtype=float32)

In [ ]:
#!pip install tensorflowjs

In [20]:
!tensorflowjs_converter --input_format keras 'modelAsli/model1.h5' 'model'

Traceback (most recent call last):
  File "c:\users\user\anaconda2\envs\py3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\user\anaconda2\envs\py3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\USER\Anaconda2\envs\py3\Scripts\tensorflowjs_converter.exe\__main__.py", line 7, in <module>
  File "c:\users\user\anaconda2\envs\py3\lib\site-packages\tensorflowjs\converters\converter.py", line 735, in pip_main
    main([' '.join(sys.argv[1:])])
  File "c:\users\user\anaconda2\envs\py3\lib\site-packages\tensorflowjs\converters\converter.py", line 739, in main
    convert(argv[0].split(' '))
  File "c:\users\user\anaconda2\envs\py3\lib\site-packages\tensorflowjs\converters\converter.py", line 654, in convert
    weight_shard_size_bytes=weight_shard_size_bytes)
  File "c:\users\user\anaconda2\envs\py3\lib\site-packages\tensorflowjs\converters\converter.py", line 74, in dispatch_keras_h5_to_tfjs_layers_model_conversion

In [43]:
!tensorflowjs_converter --input_format keras D:\POST_S1\BANGKIT\TomatoLeafDisease\model.h5 model

In [35]:
modelBaru = load_model('modelAsli/model1.h5')